In [ ]:
import os, time
import exifread
import datetime
import shutil

In [ ]:
root = r'F:\GitHub\Job\临时工作\7-(10-11)-谢光旭'
print("root exist= ",os.path.exists(root))

In [ ]:
def getImgTime(path):
    with open(path, 'rb') as f:
        data_org = exifread.process_file( f )
        t = data_org['EXIF DateTimeOriginal']
        t = time.strptime(str(t), "%Y:%m:%d %H:%M:%S")
        y,m,d,H,M,S = t[0:6]
        data=datetime.datetime(y,m,d,H,M,S)
        return data

In [ ]:
def get_delta(root, fname_list, start, n = 1):
    length = len(fname_list)
    assert start < length, "Error : start bigger than length"
    cnt = 0
    avg_time = 0
    for i, j in zip(range(start, length-1), range(start+1, length)):
        if cnt == n:
            break
        f1 = os.path.join(root, fname_list[i])
        f2 = os.path.join(root, fname_list[j])
        
        delta_time = (getImgTime(f2) - getImgTime(f1)).seconds
        #print(delta_time)
        avg_time += delta_time
        cnt += 1
    avg_time = avg_time / cnt if cnt else 0
    return round(avg_time, 4)

In [ ]:
fname_list = [f for f in os.listdir(root) if f.split('.')[-1] == 'jpg']
N = len(fname_list)
delta_list = []
for i in range(0,N):
    interval = get_delta(root, fname_list, i)
    delta_list.append(interval)
print("there are %d jpg image in the folder %s"%(N, root))
print("Interval time")
print(delta_list)
threshold = 80
CP = [i+1 for i, v in enumerate(delta_list) if v > threshold]
print("Possible break point")
print(CP)

In [ ]:
def sorted_img(root, fname_list, cutted_points):
    if len(cutted_points) == 0:
        print("Warning : cutted_points is empty")
        return
    cutted_idx = 0
    
    folder_path = os.path.join(root, "%d"%cutted_idx)
    print(folder_path)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    for i, fname in enumerate(fname_list):
        if i != cutted_points[cutted_idx]:
            path = os.path.join(root, fname)
            print(path)
            new_path = os.path.join(root, folder_path, fname)
            shutil.copy(path, new_path)
        else:
            cutted_idx += 1
            if cutted_idx == len(cutted_points):
                break
            folder_path = os.path.join(root, "%d"%cutted_idx)
            if not os.path.exists(folder_path):
                os.mkdir(folder_path)
            print(folder_path)
      
print("del the last folder")
for invalid_folder in [os.path.join(root, "%d"%i) for i in range(16)]:
    if os.path.exists(invalid_folder):
        print("del folder ",invalid_folder)
        shutil.rmtree(invalid_folder)
print("Separate picutres according to break points")
sorted_img(root, fname_list, CP)

In [ ]:
print("Rename the picture in the folder by the folder name")
dir_path = r'F:\GitHub\Job\临时工作\123\7-(10-11)-谢光旭-分文件夹'
def rename_by_folder(root):
    dir_list = os.listdir(root)
    for dname in dir_list:
        dname_c = os.path.join(root, dname)
        img_list = os.listdir(dname_c)
        cnt = 0
        for img in img_list:
            img = os.path.join(dname_c, img)
            
            cnt += 1
            new_name = dname+'_'+str(cnt)+'.jpg'
            new_name = os.path.join(dname_c, new_name)
            #print(img)
            #print(new_name)
            
            os.rename(img, new_name)
            #new_path = os.path.join(root, new_name)
            #print(new_path)
            shutil.copy(new_name, root)
            
rename_by_folder(dir_path)        